federalreserve speech 크롤링
https://github.com/bendurham441/fed-nlp/blob/main/data-collection/get_speeches.ipynb참고하여 변형

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import time
import random
import math

I get the request details by copying the request to the `/json/ne-speeches` endpoint as cURL and then using [this website](https://curlconverter.com/) to convert that to a python request

In [4]:
cookies = {
    'google-analytics_v4_7d17__engagementStart': '1699382345044',
    'google-analytics_v4_7d17__counter': '377',
    'google-analytics_v4_7d17__session_counter': '32',
    'google-analytics_v4_7d17__ga4': '20e77a5d-205a-42f6-8a19-0bd0e82b9f9a',
    'google-analytics_v4_7d17__let': '1699382421247',
    'cf_clearance': 'BzBM.S_65AbLlNh4OCc4ASoJQYYBHpFte6hyoUZrEsw-1699382102-0-1-1a686270.b107120b.4d693d90-0.2.1699382102',
    'google-analytics_v4_7d17__engagementPaused': '1699382421247',
    '_cfuvid': 'ZP4rPMFRXHyfqp2Of7OxyC06wR0ULECDDdiMFmudfqs-1699131072732-0-604800000',
    '__utma': '197852984.794505660.1699131073.1699131073.1699131073.1',
    '__utmb': '197852984.1.10.1699131073',
    '__utmc': '197852984',
    '__utmz': '197852984.1699131073.1.1.utmcsr=ideas.repec.org|utmccn=(referral)|utmcmd=referral|utmcct=/',
    '__cf_bm': '04uG.qUDKAeXwWUu76SR4QkaBe.JaGPPlRIVNxLp66k-1699382086-0-AXqir9G6ifK5a10aO0gDRulBH9bVJ12LmbmFr2wwMY91K6qw5LUBy4Qm83BBTTMaZseZOCXIZu2oJhYUABOa218=',
    'google-analytics_v4_7d17__ga4sid': '2027065144',
    'BIGipServerwww.federalreserve.gov_hsts.app~www.federalreserve.gov_hsts_pool': '!Y9yC73q6/uK6A9rjGVpQzzWrPrLO+0rNrJymKFw85wbB7O9OAQrn8qBGAO6SakmNp8aadZDpFfKTsw==',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/119.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.federalreserve.gov/newsevents/speeches.htm',
    # 'Cookie': 'google-analytics_v4_7d17__engagementStart=1699382345044; google-analytics_v4_7d17__counter=377; google-analytics_v4_7d17__session_counter=32; google-analytics_v4_7d17__ga4=20e77a5d-205a-42f6-8a19-0bd0e82b9f9a; google-analytics_v4_7d17__let=1699382421247; cf_clearance=BzBM.S_65AbLlNh4OCc4ASoJQYYBHpFte6hyoUZrEsw-1699382102-0-1-1a686270.b107120b.4d693d90-0.2.1699382102; google-analytics_v4_7d17__engagementPaused=1699382421247; _cfuvid=ZP4rPMFRXHyfqp2Of7OxyC06wR0ULECDDdiMFmudfqs-1699131072732-0-604800000; __utma=197852984.794505660.1699131073.1699131073.1699131073.1; __utmb=197852984.1.10.1699131073; __utmc=197852984; __utmz=197852984.1699131073.1.1.utmcsr=ideas.repec.org|utmccn=(referral)|utmcmd=referral|utmcct=/; __cf_bm=04uG.qUDKAeXwWUu76SR4QkaBe.JaGPPlRIVNxLp66k-1699382086-0-AXqir9G6ifK5a10aO0gDRulBH9bVJ12LmbmFr2wwMY91K6qw5LUBy4Qm83BBTTMaZseZOCXIZu2oJhYUABOa218=; google-analytics_v4_7d17__ga4sid=2027065144; BIGipServerwww.federalreserve.gov_hsts.app~www.federalreserve.gov_hsts_pool=!Y9yC73q6/uK6A9rjGVpQzzWrPrLO+0rNrJymKFw85wbB7O9OAQrn8qBGAO6SakmNp8aadZDpFfKTsw==',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

response = requests.get('https://www.federalreserve.gov/json/ne-speeches.json', cookies=cookies, headers=headers)

The response text is framed by three irrelevant characters that mess up the parsing, so I ignore those prior to using the parsing package

In [7]:
speeches = json.loads(response.text[3:-3])

In [9]:
base_url = 'https://www.federalreserve.gov'

Thankfully, all of the Federal Reserve speeches during this time interval are on the website in the same exact HTML format, making scraping pretty easy. I use BeautifulSoup to grap the relevant elements, removing references to footnotes and the like, ending with an array of paragraphs.

In [12]:
def get_article_text(rel_url):
    speech_page = requests.get(base_url + rel_url)
    soup = BeautifulSoup(speech_page.text, 'html.parser')
    
    # 아래와 같이 로직 수정
    article_div = soup.find('div', {'id': 'article'})
    if article_div is None:
        print(f"Article div not found for URL: {base_url + rel_url}")
        return []
    
    raw_html = article_div.find_all('div')[2]
    contents = []
    
    for item in raw_html:
        if isinstance(item, bs4.element.Tag):
            if item.name == 'hr':
                break
            elif item.name == 'p':
                footnote_check = item.find_all('a')
                if footnote_check:
                    for link in footnote_check:
                        title = link.get('title')
                        if title and 'footnote' in title:
                            link.clear()
                contents.append(item.text)
    #아래 내용은 주석처리
    #contents = [item.strip().encode('ascii', 'namereplace').decode('ascii') for item in contents]
    #contents = [item.replace('\\N{LATIN SMALL LETTER A WITH CIRCUMFLEX}\\x80\\x94', '--') for item in contents]
    contents = [{'text': item, 'index': i} for i, item in enumerate(contents)]
    return contents

In [14]:
def get_speech_range(min, max):
    records = []
    for i in range(min, max):
        speech = speeches[i]
        if speech.get('updateDate'):
            continue
        time.sleep(5 + random.random() * 5)
        print(speech['l'])
        symbols = []

        paras = get_article_text(speech['l'])
        meta = {
            'speaker': speech['s'],
            'date': speech['d'].split(' ')[0],
            'title': speech['t']
        }
        new_records = [{**para, **meta} for para in paras]

        records.extend(new_records)
    return records

In [22]:
# 100개씩 끊어서 생성
for i in range(math.ceil(len(speeches) / 100)):
    start = i * 100
    end = min(i * 100 + 100, len(speeches))
    records = get_speech_range(start, end)
    df = pd.DataFrame.from_records(records)
    # 문자열이 깨지는 현상으로 인해  encoding='utf-8-sig' 추가
    df.to_csv(f'./../data/speeches_{start}-{end}.csv', encoding='utf-8-sig', index=False)

/newsevents/speech/bowman20240724a.htm
/newsevents/speech/bowman20240718a.htm
/newsevents/speech/waller20240717a.htm
/newsevents/speech/kugler20240716a.htm
/newsevents/speech/bowman20240710a.htm
/newsevents/speech/cook20240710a.htm
/newsevents/speech/bowman20240709a.htm
/newsevents/speech/barr20240709a.htm
/newsevents/speech/bowman20240627a.htm
/newsevents/speech/bowman20240626a.htm
/newsevents/speech/cook20240625a.htm
/newsevents/speech/bowman20240625a.htm
/newsevents/speech/bowman20240625b.htm
/newsevents/speech/waller20240624a.htm
/newsevents/speech/kugler20240618a.htm
/newsevents/speech/cook20240617a.htm
/newsevents/speech/bowman20240617a.htm
/newsevents/speech/cook20240614a.htm
/newsevents/speech/cook20240607a.htm
/newsevents/speech/bowman20240528a.htm
/newsevents/speech/waller20240524a.htm
/newsevents/speech/waller20240521a.htm
/newsevents/speech/powell20240519a.htm
/newsevents/speech/barr20240520a.htm
/newsevents/speech/jefferson20240520a.htm
/newsevents/speech/waller20240520a.h

CSV files 병합

In [30]:
final_fname = './../dummy_data/speeches.csv'

for i in range(0, math.ceil(len(speeches) / 100)):
    start = i * 100
    end = min(i * 100 + 100, len(speeches))
    file = f'./../data/speeches_{start}-{end}.csv'

    df = pd.read_csv(file)
    
    df.to_csv(final_fname, mode='a',encoding='utf-8-sig', index=False)